In [1]:
import pandas as pd
import torch
from model.clamp import CLaMPLite

# Load the model
# model = CLaMPLite.load_from_checkpoint("/home/yj/PycharmProjects/MIT/CLaMP/outputs/2023-10-18/12-16-09/epoch=57-step=39904.ckpt")

/home/yj/anaconda3/envs/llamat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from torch_geometric.data import Data
from data.datamodule import graph_data_collator

In [2]:
df = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions.parquet')

In [3]:
df.keys()

Index(['atoms', 'space group symbol', 'crystal system', 'energy per atom',
       'volume', 'formation energy per atom', 'pretty formula',
       'energy above hull', 'band gap', 'density', 'total magnetization',
       'oxide type', 'scintillation attenuation length', 'enthalpy per atom',
       'gpt_text', 'gpt_explanation', 'reduced_formula', 'edge_index',
       'edge_attr', 'num_nodes', 'node_feat', 'y', 'text',
       'structure_question_list', 'composition_question_list'],
      dtype='object')

In [5]:
df.sample(1000, random_state=42, axis=0)

,atoms,space group symbol,crystal system,energy per atom,volume,formation energy per atom,pretty formula,energy above hull,band gap,density,...,enthalpy per atom,gpt_text,gpt_explanation,reduced_formula,edge_index,edge_attr,num_nodes,node_feat,y,text
80784,"{'abc': [12.829062, 12.829061, 12.82906], 'ang...",I4/mmm,tetragonal,-5.649577,353.967248,-2.085452,Sr4Ca2TlCu4PbO14,0.025462,0.0000,6.194266,...,-4.963803,The material stored in the database has the pr...,Possible application areas for this material a...,Sr4Ca2TlCu4PbO14,"[[0, 25, 0, 13, 0, 15, 0, 14, 0, 12, 0, 20, 0,...","[[0.5394935607910156, 0.5648473501205444, -2.5...",26,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material stored in the database has the pr...
29530,"{'abc': [3.73095, 3.89695, 11.54373], 'angles'...",Pmmm,orthorhombic,-6.215004,167.837964,-2.225595,Sr2YCu3O7,0.021926,0.0000,5.607550,...,-5.566585,The substance recorded in the database has the...,Sr2YCu3O7 is a high-temperature superconductor...,Sr2YCu3O7,"[[0, 8, 0, 8, 0, 8, 0, 8, 0, 10, 0, 10, 0, 11,...","[[1.865475058555603, 1.9484750032424927, 0.186...",13,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance recorded in the database has the...
105916,"{'abc': [4.87147, 4.871468, 3.9480500000000003...",Cmmm,orthorhombic,-6.119402,83.990392,-0.576009,SmSiNi4,0.000000,0.0000,8.169583,...,-6.030855,The substance with the database ID 105916 has ...,"Based on the given properties, SmSiNi4 can be ...",SmSiNi4,"[[0, 3, 0, 3, 0, 3, 0, 2, 0, 2, 0, 2, 0, 5, 0,...","[[-2.414982318878174, 1.500135898590088, 0.0],...",6,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance with the database ID 105916 has ...
13059,"{'abc': [8.80808, 8.808079, 8.56269], 'angles'...",P6_3/m,hexagonal,-7.701726,575.311893,-3.456436,CaZrAl9BO18,0.003200,4.8628,3.884629,...,-6.972740,The substance stored in the database has a hex...,"Based on the given properties, possible applic...",CaZrAl9BO18,"[[0, 40, 0, 38, 0, 50, 0, 43, 0, 48, 0, 31, 0,...","[[-0.37490710616111755, 2.2852938175201416, 0....",60,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance stored in the database has a hex...
118010,"{'abc': [6.648598, 7.9197489999999995, 7.8866]...",P1,triclinic,-7.208142,378.959855,-2.708520,Li5Ti2V3O10,0.011569,0.9222,3.884561,...,-6.705832,The substance with database ID 118010 has the ...,The substance with database ID 118010 is an in...,Li5Ti2V3O10,"[[0, 24, 0, 27, 0, 33, 0, 38, 0, 37, 0, 34, 0,...","[[-1.9656203985214233, -0.5153701305389404, -0...",40,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance with database ID 118010 has the ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64707,"{'abc': [8.2337, 8.2337, 5.56096], 'angles': [...",P4_2/mcm,tetragonal,-7.716994,376.998952,-3.078753,SiO2,0.199244,4.8008,2.117191,...,-6.311424,This material is a tetragonal crystal structur...,"Based on the given properties, here are some p...",SiO2,"[[0, 10, 0, 13, 0, 22, 0, 8, 0, 7, 0, 5, 0, 5,...","[[-0.6370907425880432, 0.5665197372436523, -1....",24,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],This material is a tetragonal crystal structur...
9420,"{'abc': [7.520636, 7.520637, 7.52064], 'angles...",I-42d,tetragonal,-5.151534,327.141066,-1.609517,Dy2MgSe4,0.075999,1.7730,6.752434,...,-5.361148,The substance is a material with the pretty fo...,Possible application areas for Dy2MgSe4 includ...,Dy2MgSe4,"[[0, 9, 0, 13, 0, 10, 0, 6, 0, 8, 0, 12, 0, 11...","[[-2.215404510498047, 0.7823954224586487, -1.5...",14,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance is a material with the pretty fo...
98782,"{'abc': [5.413161, 24.23484, 5.41317], 'angles...",P6_422,hexagonal,-6.880467,614.996259,-2.587428,LiFeSiO4,0.038306,3.0831,2.508954,...,-6.054486,The material with the ID number 98782 has a ch...,"Based on the material properties, possible app...",LiFeSiO4,"[[0, 39, 0, 21, 0, 26, 0, 32, 0, 16,

In [40]:
# train/test/valid split 나눠서 공개해야 할듯
crystal_question_list = []
for i, row in df.iloc[:].iterrows():
    crystal_questions = [f'This material is {row["crystal system"]}.']
    for crys in ['tetragonal', 'hexagonal', 'orthorhombic', 'cubic', 'trigonal', 'monoclinic', 'triclinic']:
        if row['crystal system'] != crys:
            crystal_questions.append(f'This material is {crys}.')
    crystal_question_list.append(crystal_questions)

In [33]:
len(set(row['atoms']['elements']))

4

In [35]:
from data.data_generate import periodic_table

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
list(periodic_table.values())

In [39]:
num_elements = []
for row in df.iloc:
    num_elements.append(len(set(row['atoms']['elements'])))

    # sample random 
max(num_elements)

9

In [48]:
p_t_v = list(periodic_table.values())

In [55]:
import numpy as np
np.random.randint(1, 10)
f'{np.random.choice(p_t_v, np.random.randint(1, 10))}'
from tqdm import tqdm

In [54]:
f'This material is composed of {[periodic_table[k] for k in set(row["atoms"]["elements"])]}.'

"This material is composed of ['Arsenic', 'Oxygen', 'Iron', 'Zinc']."

In [58]:
composition_question_list = []
for i, row in tqdm(df.iloc[:].iterrows()):
    composition_questions = [f'This material is composed of {[periodic_table[k] for k in set(row["atoms"]["elements"])]}.']
    # sample random from 1 to 9
    for _ in range(9):
        random_num = np.random.randint(1, 10)
        random_elements = np.random.choice(p_t_v, random_num)
        composition_questions.append(f'This material is composed of {set(random_elements)}.')
    composition_question_list.append(composition_questions)

126335it [01:23, 1513.74it/s]


In [60]:
composition_question_list[0]

["This material is composed of ['Copper', 'Bromine', 'Antimony', 'Cesium'].",
 "This material is composed of {'Argon', 'Roentgenium', 'Gallium', 'Seaborgium', 'Holmium', 'Manganese', 'Tungsten', 'Rhodium', 'Aluminum'}.",
 "This material is composed of {'Strontium', 'Holmium', 'Osmium', 'Erbium', 'Platinum', 'Cadmium'}.",
 "This material is composed of {'Oganesson', 'Einsteinium', 'Thulium', 'Nihonium'}.",
 "This material is composed of {'Selenium', 'Flerovium', 'Promethium'}.",
 "This material is composed of {'Antimony', 'Gallium', 'Sulfur', 'Indium', 'Oxygen'}.",
 "This material is composed of {'Krypton', 'Flerovium', 'Polonium', 'Nobelium', 'Beryllium', 'Moscovium', 'Radium', 'Nickel'}.",
 "This material is composed of {'Phosphorus', 'Livermorium'}.",
 "This material is composed of {'Chlorine', 'Lanthanum', 'Scandium', 'Nobelium', 'Protactinium', 'Silicon', 'Bismuth', 'Curium'}.",
 "This material is composed of {'Beryllium'}."]

In [4]:
# jarvis mp
from jarvis.db.figshare import data as jdata

mp_3d = pd.DataFrame(jdata('mp_3d_2020'))
mp_3d.keys()

Obtaining Materials Project-3D CFID dataset 127k...
Reference:https://doi.org/10.1063/1.4812323
Loading the zipfile...
Loading completed.


Index(['id', 'energy', 'energy_per_atom', 'volume',
       'formation_energy_per_atom', 'nsites', 'unit_cell_formula',
       'pretty_formula', 'is_hubbard', 'elements', 'nelements', 'e_above_hull',
       'hubbards', 'is_compatible', 'spacegroup', 'task_ids', 'band_gap',
       'density', 'icsd_id', 'icsd_ids', 'total_magnetization', 'material_id',
       'oxide_type', 'tags', 'elasticity', 'piezo', 'diel', 'full_formula',
       'atoms', 'desc'],
      dtype='object')

In [14]:
# train/test/valid split 나눠서 공개해야 할듯
metal_question_list = []
for i, row in df.iloc[:].iterrows():
    if 'metal' in row['text'].lower():
        metal_questions = ['This material is a metal.', 'This material is not a metal.']
        metal_question_list.append(metal_questions)
    else:
        metal_questions = ['This material is not a metal.', 'This material is a metal.']
        metal_question_list.append(metal_questions)

In [17]:
semiconductor_question_list = []
for i, row in df.iloc[:].iterrows():
    if 'semiconductor' in row['text'].lower():
        semiconductor_questions = ['This material is a semiconductor.', 'This material is not a semiconductor.']
        semiconductor_question_list.append(semiconductor_questions)
    else:
        semiconductor_questions = ['This material is not a semiconductor.', 'This material is a semiconductor.']
        semiconductor_question_list.append(semiconductor_questions)

In [29]:
stable_question_list = []
for i, row in df.iloc[:].iterrows():
    if 'not stable' in row['text'].lower() or 'unstable' in row['text'].lower():
        stable_questions = ['This material is unstable.', 'This material is stable.']
        stable_question_list.append(stable_questions)
    else:
        stable_questions = ['This material is stable.', 'This material is unstable.']
        stable_question_list.append(stable_questions)

In [38]:
# train/test/valid split 나눠서 공개해야 할듯
oxide_question_list = []
for i, row in df.iloc[:].iterrows():
    if row['oxide type'] == 'None':
        oxide_questions = ['This material is not an oxide.']
    else:
        oxide_questions = [f'This material is {row["oxide type"]}.']
    for oxide in ['None', 'oxide', 'superoxide', 'hydroxide', 'ozonide', 'peroxide']:
        if row['oxide type'] != oxide:
            if row['oxide type'] == 'None':
                oxide_questions.append('This material is not an oxide.')
            else:
                oxide_questions.append(f'This material is not {oxide}.')
    oxide_question_list.append(oxide_questions)

In [41]:
df['structure_question_list'] = crystal_question_list
# df['composition_question_list'] = composition_question_list
df['metal_question_list'] = metal_question_list
df['semiconductor_question_list'] = semiconductor_question_list
df['stable_question_list'] = stable_question_list
df['oxide_question_list'] = oxide_question_list

In [42]:
df['is_metal'] = df['text'].apply(lambda x: 'metal' in x.lower())
df['is_semiconductor'] = df['text'].apply(lambda x: 'semiconductor' in x.lower())
df['is_stable'] = df['text'].apply(lambda x: not ('not stable' in x.lower() or 'unstable' in x.lower()))

In [43]:
df.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions.parquet')

In [2]:
import pandas as pd
df = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions.parquet')
df

,atoms,space group symbol,crystal system,energy per atom,volume,formation energy per atom,pretty formula,energy above hull,band gap,density,...,text,structure_question_list,composition_question_list,metal_question_list,semiconductor_question_list,stable_question_list,is_metal,is_semiconductor,is_stable,oxide_question_list
0,"{'abc': [7.833244, 7.833245, 7.83325], 'angles...",Fm-3m,cubic,-3.026985,339.868503,-1.042928,Cs2CuSbBr6,0.037415,0.4582,4.546463,...,Cs2CuSbBr6 is a cubic crystalline material wit...,"[This material is cubic., This material is tet...","[This material is composed of ['Copper', 'Brom...","[This material is not a metal., This material ...","[This material is a semiconductor., This mater...","[This material is stable., This material is un...",False,True,True,"[This material is not an oxide., This material..."
1,"{'abc': [2.93071, 2.93071, 2.93071], 'angles':...",Pm-3m,cubic,-5.957301,25.171970,-0.140069,TiBe,0.005377,0.0000,3.752194,...,The substance with the database entry listed a...,"[This material is cubic., This material is tet...","[This material is composed of ['Titanium', 'Be...","[This material is a metal., This material is n...","[This material is not a semiconductor., This m...","[This material is stable., This material is un...",True,False,True,"[This material is not an oxide., This material..."
2,"{'abc': [4.10291, 5.36576, 6.94965], 'angles':...",Pnma,orthorhombic,-5.629226,152.998194,-0.501786,TmNi,0.000000,0.0000,9.882065,...,The substance is a crystalline material with a...,"[This material is orthorhombic., This material...","[This material is composed of ['Nickel', 'Thul...","[This material is not a metal., This material ...","[This material is not a semiconductor., This m...","[This material is stable., This material is un...",False,False,True,"[This material is not an oxide., This material..."
3,"{'abc': [3.030575, 6.390968, 12.03601], 'angle...",P1,triclinic,-5.817627,218.729028,-1.860331,Li7Mn2(CoO4)3,0.127803,0.2377,4.002799,...,The substance is a solid material with the che...,"[This material is triclinic., This material is...","[This material is composed of ['Oxygen', 'Coba...","[This material is not a metal., This material ...","[This material is not a semiconductor., This m...","[This material is unstable., This material is ...",False,False,False,"[This material is oxide., This material is not..."
4,"{'abc': [8.411952, 8.411952, 8.411950000000001...",R3,trigonal,-6.749444,432.054775,-2.047105,Mn3CoNi2(PO4)6,0.199451,0.0000,3.501150,...,The substance with the pretty formula Mn3CoNi2...,"[This material is trigonal., This material is ...","[This material is composed of ['Nickel', 'Coba...","[This material is not a metal., This material ...","[This material is not a semiconductor., This m...","[This material is unstable., This material is ...",False,False,False,"[This material is oxide., This material is not..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126330,"{'abc': [5.17955, 5.179548, 15.91706], 'angles...",Cmcm,orthorhombic,-5.416528,238.120897,-0.574610,Fe2O3,1.332748,0.0000,4.454351,...,The substance in question is an orthorhombic m...,"[This material is orthorhombic., This material...","[This material is composed of ['Oxygen', 'Iron...","[This material is a metal., This material is n...","[This material is not a semiconductor., This m...","[This material is unstable., This material is ...",True,False,False,"[This material is oxide., This material is not..."
126331,"{'abc': [7.898652, 7.898655, 17.29874], 'angle...",C2,monoclinic,-6.171699,691.809324,-1.918218,Bi6P2O15,0.104265,0.0000,7.468825,...,The material with the formula Bi6P2O15 has a c...,"[This material is monoclinic., This material i...","[This material is composed of ['Bismuth', 'Pho...","[This material is not a metal., This material ...","[This material is not a semiconductor., This m...","[This material is unstable., This material is ...",False,False,False,"[This material

In [3]:
df['oxide type'].unique()

array(['None', 'oxide', 'superoxide', 'hydroxide', 'ozonide', 'peroxide'],
      dtype=object)

In [44]:
# train/test/valid split 
df_train = df.iloc[:int(len(df)*0.8)]
df_valid = df.iloc[int(len(df)*0.8):int(len(df)*0.9)]
df_test = df.iloc[int(len(df)*0.9):]

df_train.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_train.parquet')
df_valid.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_val.parquet')
df_test.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_test.parquet')

In [27]:
import pandas as pd
df_gpt_train = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_train.parquet')
df_gpt_valid = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_val.parquet')
df_gpt_test = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_test.parquet')

df_paper_train = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_cif_papers_train.parquet')
df_paper_valid = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_cif_papers_val.parquet')
df_paper_test = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_cif_papers_test.parquet')

In [28]:
# merge train
df_train = pd.concat([df_gpt_train, df_paper_train])
df_train.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_merged_train.parquet')
df_valid = pd.concat([df_gpt_valid, df_paper_valid])
df_valid.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_merged_val.parquet')
df_test = pd.concat([df_gpt_test, df_paper_test])
df_test.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_merged_test.parquet')